# Time Series Analysis

Kasus ini akan melakukan time series analysis dari dataset bernama Daily Climate Series.
Link Dataset: https://www.kaggle.com/datasets/sumanthvrao/daily-climate-time-series-data

Kasusnya adalah ingin memprediksi cuaca.

Notes: 
Belum ada info diawalain itu memprediksi di satu langkah kedepan (1 hari) atau beberapa langkah kedepan

# Importing Library

In [1]:
import numpy as np
import pandas as pd
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import tensorflow as tf

/Users/dicoding/Dicoding/github/mlops/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Importing Dataset

In [2]:
df = pd.read_csv("https://github.com/ridhaginanjar/mlops-explore/releases/download/household-time-series/household_power_consumption.csv", sep=',', infer_datetime_format=True, index_col='datetime', header=0)
df.head()

/var/folders/8j/v2ynd5256ylfs21m0cbfj50h0000gp/T/ipykernel_27337/2880810592.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv("https://github.com/ridhaginanjar/mlops-explore/releases/download/household-time-series/household_power_consumption.csv", sep=',', infer_datetime_format=True, index_col='datetime', header=0)


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


# Cleaning Dataset

## Normalization

In [3]:
# Normalization
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

data = df.values
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

## Feature Selection

In [4]:
data

array([[0.43377912, 0.47826087, 0.04036551, ..., 0.        , 0.01282051,
        0.85      ],
       [0.55716135, 0.49885584, 0.0355582 , ..., 0.        , 0.01282051,
        0.8       ],
       [0.55867127, 0.56979405, 0.03420739, ..., 0.        , 0.02564103,
        0.85      ],
       ...,
       [0.03710095, 0.        , 0.05983313, ..., 0.        , 0.        ,
        0.        ],
       [0.03559103, 0.        , 0.06515693, ..., 0.        , 0.        ,
        0.        ],
       [0.03774806, 0.        , 0.06730234, ..., 0.        , 0.01282051,
        0.        ]])

In [5]:
N_FEATURES = len(df.columns)
SPLIT_TIME = int(len(data) * 0.5)
x_train = data[:SPLIT_TIME]
x_valid = data[SPLIT_TIME:]

In [ ]:
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series) # Mengubah series menjadi dataset tensorflow
    ds = ds.window(n_past + n_future, shift=shift, drop_remainder=True) # Melakukan windowing (dalam kasus ini 60 +1)
    ds = ds.flat_map(lambda x:x.batch(n_past + n_future)) # Melakukan flattening -> e.g: [[1],[2],[3],[4],[5]] menjadi [1,2,3] [2,3,4] [3,4,5]
    ds = ds.map(lambda x: (x[:n_past], x[n_future:])) # memecah x dan y -> x = [[2,3]] y =[[4]]
    return ds.batch(batch_size).prefetch(1) # mengelompokkan per batch -> e.g: batch 1 x = [[2,3]] y =[[4]], batch 2 = xxxx 

## Splitting Data

In [7]:
BATCH_SIZE = 32
N_PAST = 24
N_FUTURE = 24
SHIFT = 1

train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE, n_past=N_PAST, n_future=N_FUTURE, shift=SHIFT)
valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE, n_past=N_PAST, n_future=N_FUTURE, shift=SHIFT)

# Training

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, input_shape=(N_PAST, N_FEATURES)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(N_FEATURES)
])

/Users/dicoding/Dicoding/github/mlops/.venv/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Define Callbacks

In [9]:
class myCallBacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('mae') < 0.555 and logs.get('val_mae') < 0.555):
            self.model.stop_training = True

callbacks = myCallBacks()

In [10]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss='mae', optimizer=optimizer, metrics=["mae"])

## Training

In [11]:
history = model.fit(train_set, validation_data=(valid_set), epochs=100, callbacks=callbacks, verbose=1)

Epoch 1/100
   1318/Unknown 2s 1ms/step - loss: 0.0744 - mae: 0.0744

2025-05-09 13:54:34.816728: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/dicoding/Dicoding/github/mlops/.venv/lib/python3.9/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1349/1349 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0742 - mae: 0.0742 - val_loss: 0.0602 - val_mae: 0.0602


2025-05-09 13:54:36.445868: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


# Evaluate

In [12]:
train_pred = model.predict(train_set)
train_pred[0][0]

1349/1349 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


array([ 0.2658706 ,  0.12326753,  0.04302836,  0.28176   , -0.011466  ,
       -0.00538698,  0.88532746], dtype=float32)